![TS banner](banner.png)

# A JWST NIRSpec/MIRI eclipse as seen through `transitspectroscopy`
**Author**: Néstor Espinoza (Assistant Astronomer; Mission Scientist for Exoplanet Science)

**Last updated**: May 9th, 2024

## Motivation & Data Description

In this notebook, we aim to obtain a MIRI eclipse of [the exoplanet TRAPPIST-1b (PID 1177; PI Greene)](https://www.stsci.edu/jwst/science-execution/program-information?id=1177) using `transitspectroscopy`. This dataset was obtained as part of a Guaranteed Time Observations JWST program to try to detect the secondary eclipse of the planet, which was published in [Greene et al. (2023)](https://www.nature.com/articles/s41586-023-05951-7) --- here, we aim to reproduce a sort of "step by step" on how to reproduce those results, at least for a single eclipse.

As the documentation states, `transitspectroscopy` makes use of Stage 1 of the JWST Calibration Pipeline for most of its detector-level calibration. Because of this we list below the versions of the JWST pipeline and `transitspectroscopy` we will be using:

In [1]:
import jwst
import transitspectroscopy as ts

print('JWST Calibration pipeline version:', jwst.__version__)
print('transitspectroscopy version:', ts.__version__)

JWST Calibration pipeline version: 1.14.0
transitspectroscopy version: 0.4.0


And now let's load a set of helper libraries:

In [2]:
import glob

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('ticks')

<div class="alert alert-block alert-info"> <b>NOTE ON SCREAMING FACE EMOJIS (😱)</b>: In this notebook, we will use the very nice notation introduced on <a href="https://hastie.su.domains/ElemStatLearn/">"The Elements of Statistical Learning"</a> that consider screaming faces (😱) when starting some sections. Whenever you see one, it means the section can be skipped --- the content is interesting if you want a "deep dive" or "deeper knowledge" on the topic being discussed, but it is not needed to continue the tutorial.</div>

## 1. First steps: from raw data to ramps

### 1.1 Downloading the data
To start, let's download the corresponding JWST data (if you already have the data in your system, you can skip this step). `transitspectroscopy` has a neat download function that uses `astroquery` to get you the data you need. The two things one needs to get this data is **the program ID (1177)** and the **observation number (here, we use 11, which is the first eclipse that was observed)**. Let's use that to download the data:

In [ ]:
ts.jwst.download(pid = 1177, obs_num = '11')

This will take a few minutes depending on your internet connection --- but totally worth the beautiful JWST data! 

<div class="alert alert-block alert-info"> <b>Note:</b> By default, the function downloads the <code>uncal</code> data --- the uncalibrated data. This can be changed to download either ramps or rateints by using the flag <code>data_product</code> on the <code>ts.jwst.download</code> call (e.g., <code>ts.jwst.download(pid = 1177, obs_num = '11', data_product="rateints")</code> will download the rates per integration; <code>data_product="ramps"</code> downloads the calibrated ramps). </div>

In the output above, one thing to note is that there's "segments" of data, with filenames names including the words `seg001`, `seg002`, etc. Those are [data segments](https://jwst-docs.stsci.edu/getting-started-with-jwst-data/understanding-jwst-data-files/jwst-data-products): the ground mechanism for processing the data segments the entire TSO dataset into little pieces so it's easier to reduce, analyze and download the data.  

### 1.2 Detector-level calibration

Now we have the data --- let's calibrate it! First, we perform detector-level calibration using `transitspectroscopy`. By default, the library performs the standard set of steps using STScI's [JWST Calibration Pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html) with some modifications depending on the instrument. For this MIRI photometry data, the modification is in the jump-step: `transitspectroscopy` performs its own TSO-based jump detection algorithm by default.


To perform this, we first need to setup a list with the names of the files that we will be reducing. The download function above automatically downloads the data to a folder called `JWSTdata` --- but of course, you can fill below wherever you stored your `uncal` files. Let's extract filenames on lists for the dataset:

In [3]:
miri_filenames = glob.glob('/astro/planet/jwst/trappist1b-miri/1177/obs011/jw*uncal.fits')

Interesting to note: tools like `glob` don't necessarily sort the segments chronologically:

In [4]:
print(miri_filenames)

['/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg008_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg006_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg007_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg001_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg005_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg009_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg004_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg002_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg003_mirimage_uncal.fits', '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_0000

That's OK, the next function we will use, fixes this for us. 

#### 1.2.1 Loading data --- applying detector-calibration

The `transitspectroscopy` library for JWST data works by loading those datasets in an object; let's load it for this MIRI data; note this might also take a while, as this is a significant amount of data we are loading:

In [5]:
miri_dataset = ts.jwst.load(miri_filenames, outputfolder = '/astro/planet/jwst/trappist1b-miri/1177/obs011/')

	    - Instrument/Mode: MIRI/Photometry (F1500W)



This will load all the segments of data, and save any outputs in the `JWSTdata` folder (you can use any other folder, of course, to save your products!) under a new folder called `ts_outputs`. Note this sorts the filenames automatically:

In [6]:
miri_dataset.filenames

array(['/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg001_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg002_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg003_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg004_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg005_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg006_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg007_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg008_mirimage_uncal.fits',
       '/astro/planet/jwst/trappist1b-miri/1177/obs011/jw01177011001_03101_00001-seg009_mirimage_uncal.fits',
       '/a

All right! Let's perform some detector-level calibration. Again, this will take a while --- this functions does the heavy-lifting of calibrating the data from detector systematics, after all!

In [ ]:
miri_dataset.detector_calibration()

	 [START] Detector-level Calibration


	 >> Processing 10 files.

	    - TSO total duration: 4.3 hours
	    - Calibration parameters:
{'dq_init': {}, 'saturation': {}, 'refpix': {}, 'linearity': {'output_dir': '/astro/planet/jwst/trappist1b-miri/1177/obs011/ts_outputs', 'save_results': True, 'suffix': 'linearitystep'}, 'dark_sub': {}, 'jump': {'window': 10, 'nsigma': 10}, 'ramp_fit': {'output_dir': '/astro/planet/jwst/trappist1b-miri/1177/obs011/ts_outputs', 'save_results': True, 'suffix': 'ramp_fitstep'}, 'tracing': {}, 'centroiding': {}, 'reset': {}, 'emicorr': {}, 'group_1f': {}, 'skip': {}}
	 >> Linearity files found. Loading them...

	    - Instrument/Mode: MIRI/Photometry (F1500W)



2024-05-09 23:47:16,600 - stpipe.DarkCurrentStep - INFO - DarkCurrentStep instance created.
2024-05-09 23:47:16,812 - stpipe.DarkCurrentStep - INFO - Step DarkCurrentStep running with args (<RampModel(40, 14, 1024, 1032) from jw01177011001_03101_00001-seg001_mirimage_linearitystep.fits>,).
2024-05-09 23:47:16,814 - stpipe.DarkCurrentStep - INFO - Step DarkCurrentStep parameters are:
  pre_hooks: []
  post_hooks: []
  output_file: None
  output_dir: None
  output_ext: .fits
  output_use_model: False
  output_use_index: True
  save_results: False
  skip: False
  suffix: None
  search_output_file: True
  input_dir: ''
  dark_output: None
  average_dark_current: None
2024-05-09 23:47:16,838 - stpipe.DarkCurrentStep - INFO - Using DARK reference file /grp/crds/cache/references/jwst/jwst_miri_dark_0094.fits
2024-05-09 23:47:34,308 - stpipe.DarkCurrentStep - WARNING - Keyword RESERVED_4 does not correspond to an existing DQ mnemonic, so will be ignored
2024-05-09 23:47:34,340 - stpipe.DarkCur

	 >> Performing TSO-jump...



Let's explore what happened to this `miri_dataset` object --- first, note the data properties. The `miri_dataset` has a `ramps` array which holds the calibrated ramps:

In [ ]:
miri_dataset.ramps.shape

As can be seen, all segments add up to 1139 integrations, 20 groups each --- on a 32 x 2048 pixel subarray. If for some reason you wanted the ramps for each segment, you can inspect the `ramps_per_segment` **object**, which is actually a `jwst.RampModel` per segment:

In [ ]:
len(miri_dataset.ramps_per_segment)

In [ ]:
type(miri_dataset.ramps_per_segment[0])

This is neat, because that one lets you access all the properties of a `RampModel` --- like the data itself, for instance:

In [ ]:
miri_dataset.ramps_per_segment[0].data.shape

<div class="alert alert-block alert-info"> <b>Note:</b> The <code>data</code> in <code>miri_dataset.ramps_per_segment[0].data</code> is actually linked with the <code>miri_dataset.ramps</code> array for all steps of the pipeline if running them via <code>transitspectroscopy</code>. That is, changing a value in the <code>ramps_per_segment</code> will also change the corresponding <code>ramps</code> array. This is very powerful, as we will see in a moment!</div>

### 1.3 Fitting ramps

Let's now perform ramp-fitting. This uses the ramp-fitting step from the JWST pipeline directly, and applies it to our data:

In [ ]:
miri_dataset.fit_ramps()

All right! The `rateints` are stored in the same object for one to explore. As expected, we have...

In [ ]:
print(len(miri_dataset.rateints), 'rates for MIRI')

And the rateints dimension is:

In [ ]:
miri_dataset.rateints.shape

What steps of the pipeline were run? You can check this with the `status`:

In [ ]:
miri_dataset.status

Let's do a quick photometric exploration of this. First, let's check one of the rates:

In [ ]:
plt.figure(figsize=(8, 6))

plt.title('MIRI data; rates for integration 10')
im = plt.imshow(miri_dataset.rateints[10, :, :], aspect = 'auto', origin = 'lower')
im.set_clim(140,170)

plt.colorbar(label = 'Counts/s')

Let's do a close-up around TRAPPIST-1b:

In [ ]:
xcen, ycen = 697, 515
distance = 25

In [ ]:
plt.figure(figsize=(6.5, 5))

plt.title('MIRI data; rates for integration 10')
im = plt.imshow(miri_dataset.rateints[10, :, :], aspect = 'auto', origin = 'lower')
plt.plot([xcen],[ycen], 'o', ms = 10, mec = 'black', mfc = 'cornflowerblue')
plt.xlim(xcen - distance, xcen + distance)
plt.ylim(ycen - distance, ycen + distance)
im.set_clim(140,170)

plt.colorbar(label = 'Counts/s')

Let's get a quick lightcurve:

In [ ]:
lc0 = np.nansum( miri_dataset.rateints[:, ycen-distance:ycen+distance, xcen-distance:xcen+distance], axis = (1,2) )

In [ ]:
plt.figure(figsize=(8,4))

plt.title('Crude MIRI lightcurve')

plt.plot( lc0 / np.nanmedian(lc0[-100:]), 'o-', 
          color = 'cornflowerblue', mfc = 'white', ms = 2)

plt.ylim(0.996,1.0015)
plt.xlim(0.5, len(lc0)+0.5)

plt.xlabel('Integration index', fontsize = 16)
plt.ylabel('Relative flux', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.show()